In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lin = nn.Linear(5, 3)
data = autograd.Variable(torch.randn(2, 5))
print(lin(data))

Variable containing:
 0.1755 -0.3268 -0.5069
-0.6602  0.2260  0.1089
[torch.FloatTensor of size 2x3]



In [3]:
data = autograd.Variable(torch.randn(2, 2))
print(data)
print(F.relu(data))

Variable containing:
-0.5404 -2.2102
 2.1130 -0.0040
[torch.FloatTensor of size 2x2]

Variable containing:
 0.0000  0.0000
 2.1130  0.0000
[torch.FloatTensor of size 2x2]



In [8]:
data = autograd.Variable(torch.randn(5))
print(data)
print(F.softmax(data, dim=0))
print(F.softmax(data, dim=0).sum())
print(F.log_softmax(data, dim=0))

Variable containing:
 2.2820
-1.2080
 1.1120
 2.2174
-0.4269
[torch.FloatTensor of size 5]

Variable containing:
 0.4264
 0.0130
 0.1324
 0.3998
 0.0284
[torch.FloatTensor of size 5]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
-0.8523
-4.3423
-2.0222
-0.9168
-3.5611
[torch.FloatTensor of size 5]



## Logistic Refression Bag-of-Words classifier

In [9]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

In [10]:
word_to_idx = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
print(word_to_idx)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [11]:
VOCAB_SIZE = len(word_to_idx)
NUM_LABELS = 2

In [12]:
class BoWClassifier(nn.Module):
    
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=1)

In [13]:
def make_bow_vector(sentence, word_to_idx):
    vec = torch.zeros(len(word_to_idx))
    for word in sentence:
        vec[word_to_idx[word]] += 1
    return vec.view(1, -1)

In [14]:
def make_target(label, label_to_idx):
    return torch.LongTensor([label_to_idx[label]])

In [15]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [16]:
for param in model.parameters():
    print(param)

Parameter containing:

Columns 0 to 9 
-0.1816  0.0987 -0.1379 -0.1480  0.0119 -0.0334  0.1152 -0.1136 -0.1743  0.1427
-0.1268 -0.1668  0.1882  0.0102  0.1344  0.0406  0.0631  0.1465  0.1860 -0.1301

Columns 10 to 19 
-0.0291  0.1103  0.0630 -0.1471  0.0394  0.0471 -0.1313 -0.0931  0.0669  0.0351
 0.0245  0.1464  0.1421  0.1218 -0.1419 -0.1412 -0.1186  0.0246  0.1955 -0.1239

Columns 20 to 25 
-0.0834 -0.0594  0.1796 -0.0363  0.1106  0.0849
 0.1045 -0.1085 -0.1844 -0.0417  0.1130  0.1821
[torch.FloatTensor of size 2x26]

Parameter containing:
-0.1218
 0.0426
[torch.FloatTensor of size 2]



In [17]:
sample = data[0]

In [18]:
sample

(['me', 'gusta', 'comer', 'en', 'la', 'cafeteria'], 'SPANISH')

In [19]:
bow_vector = make_bow_vector(sample[0], word_to_idx)

In [20]:
log_probs = model(autograd.Variable(bow_vector))

In [21]:
print(log_probs)

Variable containing:
-1.0600 -0.4254
[torch.FloatTensor of size 1x2]



In [30]:
label_to_idx = {"SPANISH": 0, "ENGLISH": 1}

In [31]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_idx))
    log_probs = model(bow_vec)
    print(log_probs)

Variable containing:
-0.8265 -0.5755
[torch.FloatTensor of size 1x2]

Variable containing:
-0.9991 -0.4592
[torch.FloatTensor of size 1x2]



In [32]:
print(next(model.parameters())[:, word_to_idx["creo"]])

Variable containing:
1.00000e-02 *
 -2.9080
  2.4520
[torch.FloatTensor of size 2]



In [33]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [34]:
for epoch in range(100):
    for instance, label in data:
        model.zero_grad()
        
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_idx))
        target = autograd.Variable(make_target(label, label_to_idx))
        
        log_probs = model(bow_vec)
        
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

In [36]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_idx))
    log_probs = model(bow_vec)
    print(log_probs)

Variable containing:
-0.1582 -1.9222
[torch.FloatTensor of size 1x2]

Variable containing:
-2.7357 -0.0670
[torch.FloatTensor of size 1x2]



In [38]:
print(next(model.parameters())[:, word_to_idx["creo"]])

Variable containing:
 0.3894
-0.3939
[torch.FloatTensor of size 2]

